# Unfitted Finite Element Method for the Stokes Problem

In this notebook, we solve the Stokes equations on an **unfitted mesh** using the **ghost penalty stabilization** technique. This approach allows us to handle complex geometries without conforming the mesh to the domain boundary.

The Stokes equations in strong form are given by:

$$
\begin{aligned}
- \nu \Delta \mathbf{u} + \nabla p &= \mathbf{f} \quad \text{in } \Omega, \\
\text{div}(\mathbf{u}) &= 0 \quad \text{in } \Omega, \\
\mathbf{u} &= \mathbf{g} \quad \text{on } \partial\Omega,
\end{aligned}
$$

where: 
- $ \mathbf{u} $ is the velocity field,
- $ p $ is the pressure,
- $ \mathbf{f} $ is a given forcing term,
- and $ \mathbf{g} $ is the prescribed Dirichlet boundary condition.

In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
import ngsolve
from xfem import *
from xfem.lsetcurv import *

In [2]:
order = 2
maxh = 0.1
nu = 1.0

square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## Fictitious Domain and Level Set Representation

We work with a **fixed background mesh** $\widehat{\Omega} = [-1.25, 1.25]^2$, which is independent of the actual geometry.

The **physical domain** $\Omega$ is defined implicitly via a **level set function** $\phi(x, y)$, and corresponds either to the **interior** or **exterior** of a closed boundary $\Gamma = \{ \phi = 0 \}$.

- If $\phi(x, y) < 0$, the point $(x, y)$ lies **inside** $\Omega$
- If $\phi(x, y) > 0$, the point lies **outside**
- $\Gamma$ is the zero level set and represents the **interface**

This approach allows us to define complex geometries without modifying the mesh. We solve the Stokes problem on $\Omega$ using an unfitted finite element method, where the background mesh is cut by $\Gamma$, and stability is ensured using **ghost penalty stabilization**.


In [3]:
r = sqrt(x**2 + y**2)
levelset = r-1
lsetp1 = GridFunction(H1(mesh))
InterpolateToP1(levelset,lsetp1)# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
DrawDC(levelset, CF(1), CF(0), mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [4]:
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=order, threshold=0.1,discontinuous_qn=True)# Higher order level set approximation 
deformation = lsetmeshadap.CalcDeformation(levelset)
lsetp1 = lsetmeshadap.lset_p1
lsetp1 = GridFunction(H1(mesh))
InterpolateToP1(levelset,lsetp1)# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:

## Fictitious Domain Construction

The **fictitious domain** $\Omega^*$ is defined as the union of all background mesh elements that intersect the physical domain $\Omega$. That is, we consider the minimal subset of elements $\mathcal{T}^* \subset \widehat{\mathcal{T}}$ such that:

$$
\Omega \subset \bigcup_{T \in \mathcal{T}^*} T =: \Omega^*,
$$

where:
- $\widehat{\mathcal{T}}$ denotes the background mesh over $\widehat{\Omega}$,
- $\mathcal{T}^*$ is the collection of all elements that are **cut by** or **lie inside** the level set domain $\Omega$.

This extended domain $\Omega^*$ is used for the unfitted finite element formulation. Integration and stabilization terms are evaluated over $\Omega^*$ instead of $\Omega$, which avoids the need for boundary-fitted meshes.


In [5]:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
hasif = ci.GetElementsOfType(IF)
neg = ci.GetElementsOfType(NEG)
cut_facets = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasif)
stab_facets = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasneg)
hasneg_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg, use_and=True)

#Integration domains
dX = dCut(lsetp1, NEG, definedonelements=hasneg,deformation=deformation)
ds = dCut(lsetp1, IF, definedonelements=hasif,deformation=deformation)
dw = dFacetPatch(definedonelements=stab_facets,deformation=deformation)
dz = dFacetPatch(definedonelements=cut_facets, deformation=deformation)
dxbar = dx(definedonelements=hasneg, deformation=deformation)
dxcut = dx(definedonelements=hasif, deformation=deformation)
dxinner = dx(definedonelements=neg, deformation=deformation)
dcutskel = dCut(lsetp1, NEG, skeleton=True, definedonelements=hasneg_facets, deformation=deformation)

## Spaces for the unstabilized Formulation
For the interior penalty term we need:
$$
\mathcal{F}_h = \left\{ F \cap \Omega \; : \; F \in \partial_i \mathcal{T}^* \right\}
$$
For the Lagrange Parameter we need:
$$
\mathcal{T}_{cut} = \left\{ T \in  \mathcal{T}^*  \; : \; T \cap \Gamma \neq \emptyset \right\}
$$

## Weak Formulation using BDM Elements

We now reformulate the Stokes problem in its **weak (variational) form**. To this end, we use the **$BDM$** finite element for the velocity and **$H1$** spaces for the pressure. Since the BDM is not a subspace of H1, because the space does not provide continuity, we have to use as in the fitted case interior penalty terms. The Lagrange multiplier space is 

We seek a pair of functions $(u_h, p_h, \lambda_h) \in V_h \times Q_h \times \Lambda_h$, such that

$$
A_h(u_h, p_h, \lambda_h; v_h, q_h, \mu_h) = L_h(v_h, q_h, \mu_h) \quad \text{for all } (v_h, q_h, \mu_h) \in V_h \times Q_h \times \Lambda_h,
$$

with $V_h \times Q_h \times \Lambda_h= BDM^k \times H_1^{k-1}(\Omega) \times H_1^{k}(\mathcal{T}_{cut}) $, and where the bilinear and linear forms are defined by:

$$ \
A_h(u_h, p_h; v_h, q_h) = a_h(u_h, v_h) + b_h(u_h, q_h) + b_h(v_h, p_h) + c_h(u_h; \mu_h) + c_h(v_h; \lambda_h),
$$

$$
L_h(v_h, q_h) = (f, v_h) + \gamma h^{-1} (g,v_h)_\Gamma + (g \cdot n, \mu_h)_\Gamma - (\partial_n v_h,g)_\Gamma.
$$

### Definitions of the bilinear forms:

- $a_h(u_h, v_h) = (\nabla u_h, \nabla v_h)_\Omega - (\partial_n u_h, v_h)_\Gamma - (\partial_n v_h, u_h)_\Gamma + \gamma h^{-1} (u_h, v_h)_\Gamma - (\{\partial_n u_h\},[ v_h])_{\mathcal{F}_h} - (\{\partial_n v_h\},[ u_h])_{\mathcal{F}_h}  + \gamma h^{-1} ([u_h],[ v_h])_{\mathcal{F}_h} $  
- $b_h(v_h, p_h) = -(\nabla v_h, p_h)_\Omega$
- $c_h(u_h; \mu_h) = (u_h \cdot n, \mu_h)_\Gamma + h(\partial_n \lambda_h , \partial_n \mu_h)_{\mathcal{T}_{cut}}$

Here, $f$ is the given forcing term, and $\gamma$ is the Nitsche penalty parameter. 


In [6]:
def jump(f):
    return f - f.Other()
def avgdnF(f):
    return 0.5 * (Grad(f)*nF + Grad(f.Other())*nF)
def avg(f):
    return 0.5 * (f + f.Other())

In [7]:
gamma_stab_N_vol = lambda k: 0.01 #* (k+1)**2

In [8]:
#function spaces 
Shsbase = HDiv(mesh, order=order, dirichlet=[], dgjumps=True)
Vhbase = L2(mesh, order=order-1, dirichlet=[], dgjumps=True)
Vh = Restrict(Vhbase, hasneg)
Shs = Restrict(Shsbase, hasneg)
Fhbase = H1(mesh, order=order, dirichlet=[], dgjumps=True)
Fh = Restrict(Fhbase, hasif)
Nh = NumberSpace(mesh)
X = Shs*Vh*Fh*Nh

(u,p,lam,r),(v,q,mu,s) = X.TnT()
gfu = GridFunction(X)

h = specialcf.mesh_size
n = Normalize(grad(lsetp1))
nF= specialcf.normal(mesh.dim)        

gamma_Nitsche = 20
gamma_IP = 20

#Bilinearform
a = BilinearForm(X, symmetric=False)
a += nu* InnerProduct(Grad(u), Grad(v))*dX +  nu*(-Grad(u)*n * v -Grad(v)*n * u) * ds + nu * gamma_Nitsche / h * u * v * ds # a terms
a += -div(u) * q * dX - div(v) * p * dX #b terms
a += nu*(-avgdnF(u) * jump(v) + -avgdnF(v) * jump(u) + gamma_IP / h * jump(u) * jump(v)) * dcutskel# interior penalty terms

# Lagrange multiplier term to fix the normal velocity at the boundary
a += (u*n * mu + v*n * lam) * ds 
a += -gamma_stab_N_vol(order)*h*(grad(lam)*n) * (grad(mu)*n)*dxcut

# regularisation (to help the linear solver)
a += -1e-8 *r*s * dX - 1e-8 * p*q*dxbar

In [9]:
rhs = LinearForm(X) 
rhs += InnerProduct(f, v)* dX
rhs += uexact * n * mu * ds
rhs += uexact * nu * ( gamma_Nitsche / h * v - Grad(v)*n) * ds
rhs += pexact * s * dX

a.Assemble()
rhs.Assemble()

gfu.vec.data = a.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky") * rhs.vec

gfvel,gfpres,_,_ = gfu.components
error = sqrt(Integrate((InnerProduct(gfvel - uexact ,gfvel - uexact))*dX, mesh))
perror = sqrt(Integrate((gfpres - pexact)**2*dX, mesh))
print(error, perror)

AttributeError: 'PackagePath' object has no attribute 'InnerProduct'

## Interior Facets Near the Interface

We introduce the notation $\mathcal{F}_\Gamma^*$ for the set of all **interior facets** that belong to elements **intersected by the interface** $\Gamma$. This set plays a key role in the definition of the **ghost penalty stabilization** for the velocity and the Lagrange multiplier.

Formally, we define:

$$
\mathcal{F}_\Gamma^* = \left\{ F \in \partial_i \mathcal{T}^* \; : \; T_F^+ \cap \Gamma \neq \emptyset \; \text{or} \; T_F^- \cap \Gamma \neq \emptyset \right\},
$$
$$
\mathcal{F}_\Gamma = \left\{ F \in \partial_i \mathcal{T}^* \; : \; T_F^+ \cap \Gamma \neq \emptyset \; \text{and} \; T_F^- \cap \Gamma \neq \emptyset \right\},
$$

where:
- $\mathcal{T}^*$ is the set of active (cut or inside) elements covering the physical domain $\Omega$,
- $\partial_i \mathcal{T}^*$ denotes the set of **interior facets** (i.e., shared by two elements),
- $T_F^+$ and $T_F^-$ are the two elements sharing facet $F$.

This set $\mathcal{F}_\Gamma^*$ identifies all facets in the vicinity of the interface $\Gamma$ and is used as the integration domain for ghost penalty terms.


## Stabilization

In our formulation three terms occur, which need stabilization to obtain inf-sup-stability here. We use different ideas to ensure this. First the **classical ghost penalty** which penalize jumps of normal derivatives acroos interior facets that are intersected by the geometry. Second **an extendend operator** that simply stabilize by ignoring cuts on cutted elements. This however comes with the cost of incosistency. 

- The **velocity ghost penalty** is given by

$$
i_h(u, v) = \sum_{F \in \mathcal{F}_\Gamma^*}  \gamma_{GP} h^{-2}_F(u_a-u_b,v_a-v_b)_{\omega_F}   
$$

- The **pressure stabilization** is given by

$$
b_h(v_h, p_h) = -(\nabla v_h, p_h)_{\Omega^*}
$$

- The **Lagrange multiplier ghost penalty** is given by
$$
j_h(\lambda,\mu) = \sum_{F \in \mathcal{F}_\Gamma} \gamma_{LG} h^{-1}_F (\lambda_a-\lambda_b,\mu_a-\mu_b)_{\omega_F} 
$$

Here:

- $\mathcal{F}_\Gamma^*$ is the set of interior facets belonging to elements cut by the boundary $\Gamma$ (as defined above),
- $\omega_F$ denotes the patch of the two elements $T_a$ and $T_b$ sharing the facet F 
- $h_F$ is a measure of the facet size,
- and $\gamma_{GP}$, $\gamma_{LG}$ are positive stabilization parameters.

These terms are added to the bilinear form to improve robustness and ensure well-posedness in the unfitted setting.


In [ ]:
gamma_GP = 0.2
gamma_stab_N_fac = lambda k: 0.01 #* (k+1)**2#0.25 * (k+1)**2
#velocity ghost penalty
a += gamma_GP/h**2 * nu * InnerProduct(jump(u), jump(v)) * dw 
#b term stabiilization
a += div(u) * q * dX + div(v) * p * dX #b subtract old  
a += -div(u) * q * dxbar - div(v) * p * dxbar # add new 
#Lagrange multiplier ghost penalty
a += -gamma_stab_N_fac(order)*1/h*jump(lam)*jump(mu)* dz 

a.Assemble()
rhs.Assemble()

gfu.vec.data = a.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky") * rhs.vec

gfvel,gfpres,_,_ = gfu.components
error = sqrt(Integrate((InnerProduct(gfvel - uexact ,gfvel - uexact))*dX, mesh))
perror = sqrt(Integrate((gfpres - pexact)**2*dxinner, mesh))
print(error, perror)